In [9]:
# %%
import requests
#import mysql.connector
import os
from dotenv import load_dotenv
#import json
#import pandas as pd
import pymysql
import time

In [ ]:
# Configuración API
API_BASE_URL = "https://cramer.buk.cl/api/v1/chile/employees/absences/licences"
TOKEN = "JVZhehjdRWdmx2reD5mkKRH"

API_ENDPOINTS = {
    "licences": f"{API_BASE_URL}licences",
    "absences": f"{API_BASE_URL}absences",
    "permissions": f"{API_BASE_URL}permissions"
}

# Configuración BD
DB_HOST = "localhost"
DB_USER = "root" #REEMPLAZAR USUARIO CREADO POR GABRIEL Y CONTRASEÑA
DB_PASSWORD = os.getenv("clave_sql") #REEMPLAZAR CONTRASEÑA CREADA POR GABRIEL
DB_NAME = "datos_buk_rrhh" #REEMPLAZAR NOMBRE DE BASE DE DATOS CREADA POR GABRIEL

# Imprimir las variables de entorno
print(f"TOKEN: {os.getenv('BUK_AUTH_TOKEN')}")
print(f"SQL: {os.getenv('clave_sql')}")

TOKEN: None
SQL: None


In [17]:
# %%
def obtener_todos_los_empleados():
    """
    Función que obtiene todos los empleados iterando por todas las páginas
    """
    headers = {"auth_token": TOKEN}
    todos_los_empleados = []
    url_actual = API_BASE_URL
    pagina_actual = 1

    print("🚀 Comenzando la obtención de empleados con paginación...")

    while url_actual:
        print(f"📄 Obteniendo página {pagina_actual}...")

        respuesta = requests.get(url_actual, headers=headers)

        if respuesta.status_code == 200:
            respuesta_api = respuesta.json()
            empleados_pagina = respuesta_api['data']
            pagination_info = respuesta_api['pagination']

            # Agregar empleados de esta página a la lista total
            todos_los_empleados.extend(empleados_pagina)

            print(f"✅ Página {pagina_actual}: {len(empleados_pagina)} empleados obtenidos")
            print(f"📊 Total acumulado: {len(todos_los_empleados)} empleados")
            print(f"📈 Páginas restantes: {pagination_info['total_pages'] - pagina_actual}")

            # Verificar si hay una página siguiente
            url_actual = pagination_info.get('next')
            pagina_actual += 1

            # Pequeña pausa para no sobrecargar la API
            time.sleep(0.5)

        else:
            print(f"❌ Error en la petición página {pagina_actual}:", response.status_code)
            break

    print(f"🎉 ¡Paginación completada! Total de empleados obtenidos: {len(todos_los_empleados)}")
    return todos_los_empleados

# Obtener todos los empleados
empleados = obtener_todos_los_empleados()

🚀 Comenzando la obtención de empleados con paginación...
📄 Obteniendo página 1...
✅ Página 1: 25 empleados obtenidos
📊 Total acumulado: 25 empleados
📈 Páginas restantes: 136
📄 Obteniendo página 2...
✅ Página 2: 25 empleados obtenidos
📊 Total acumulado: 50 empleados
📈 Páginas restantes: 135
📄 Obteniendo página 3...


KeyboardInterrupt: 

In [ ]:

print("🚀 Conectando a MySQL...")
conexion = pymysql.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD,
    charset='utf8mb4'
)
cursor = conexion.cursor()

# Crear base de datos si no existe
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {DB_NAME}")
cursor.execute(f"USE {DB_NAME}")
print(f"✅ Conectado a MySQL y usando la base: {DB_NAME}")

# %%
print("🚀 Creando tabla empleados (si no existe)...")
cursor.execute("""
CREATE TABLE IF NOT EXISTS empleados (
    id INT PRIMARY KEY,
    first_name VARCHAR(100),
    surname VARCHAR(100),
    full_name VARCHAR(200),
    rut VARCHAR(20),
    email VARCHAR(200),
    phone VARCHAR(50)
)
""")
print("✅ Tabla 'empleados' lista.")


# %%
print("🚀 Insertando TODOS los empleados en la tabla SQL...")
contador = 0
errores = 0

for e in empleados:
    try:
        sql = """
        INSERT INTO empleados (id, first_name, surname, full_name, rut, email, phone)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            first_name=VALUES(first_name),
            surname=VALUES(surname),
            full_name=VALUES(full_name),
            rut=VALUES(rut),
            email=VALUES(email),
            phone=VALUES(phone)
        """
        values = (
            e.get("id"),
            e.get("first_name"),
            e.get("surname"),
            e.get("full_name"),
            e.get("rut"),
            e.get("email"),
            e.get("phone")
        )
        cursor.execute(sql, values)
        contador += 1

        # Mostrar progreso cada 100 registros
        if contador % 100 == 0:
            print(f"📝 Procesados {contador} empleados...")

    except Exception as error:
        print(f"⚠️ Error insertando empleado {e.get('id', 'N/A')}: {error}")
        errores += 1

conexion.commit()
print(f"✅ {contador} registros insertados/actualizados en MySQL.")
if errores > 0:
    print(f"⚠️ Se produjeron {errores} errores durante la inserción.")

# %%
print("🚀 Consultando estadísticas de la tabla...")
cursor.execute("SELECT COUNT(*) as total FROM empleados;")
total_registros = cursor.fetchone()[0]
print(f"📊 Total de empleados en la base de datos: {total_registros}")

print("\n🔍 Mostrando los primeros 5 empleados:")
cursor.execute("SELECT id, full_name, email FROM empleados LIMIT 5;")
for row in cursor.fetchall():
    print(f"ID: {row[0]}, Nombre: {row[1]}, Email: {row[2]}")

print("\n🔍 Mostrando los últimos 5 empleados:")
cursor.execute("SELECT id, full_name, email FROM empleados ORDER BY id DESC LIMIT 5;")
for row in cursor.fetchall():
    print(f"ID: {row[0]}, Nombre: {row[1]}, Email: {row[2]}")

# %%
cursor.close()
conexion.close()
print("✅ Conexión cerrada correctamente.")